# Airbnb Pricing Prediction: Milestone 3
**James Gearheart**<br>
**Danny Zhuang**<br>
**Bob Saludo**<br>
**Ryan Wallace**<br><br>
**Harvard University**<br>
**Fall 2016**<br>
**TF: Christine Hwang**<br>
**Due Date: ** Saturday, November 5th, 2016

---

In [ ]:
# import necessary libaries
import random
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline